# Online Tracking

Given a list of images, we want to track players and the ball and gather their trajectories. Our model initializes several tracklets based on the detected boxes in the first image. In the following ones, the model links the boxes to the existing tracklets according to: 

1. their distance measured by the embedding model,
2. their distance measured by boxes IoU's

When the entire list of image is processed, we compute an homography for each image. We then apply each homography to the players coordinates.

In [ ]:
# !pip3 install --user tensorflow==2.11.0
# !pip3 install --user tensorflow-probability==0.19.0
# !pip3 install --user dm-sonnet==2.0.0

## Inputs

Let's start by gathering a list of images: 

In [1]:
%env SM_FRAMEWORK=tf.keras

import os
import numpy as np
import torch
import torch.nn.functional as F
import mxnet as mx
import cv2
from matplotlib import pyplot as plt

import sys
sys.path.append("./narya")

from narya.utils.vizualization import visualize

from narya.tracker.full_tracker import FootballTracker

env: SM_FRAMEWORK=tf.keras


OSError: libcudnn.so.7: cannot open shared object file: No such file or directory

First, we initialize a 2d Field template:

In [ ]:
template = cv2.imread('world_cup_template.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)
template = cv2.resize(template, (1280,720))
template = template/255.

and then we create our list of images: 

In [ ]:
"""
Images are ordered from 0 to 50:
"""

imgs_ordered = []
for i in range(0,51):
    path = 'test_img/img_fullLeicester 0 - [3] Liverpool.mp4_frame_full_' + str(i) + '.jpg'
    imgs_ordered.append(path)
    
img_list = []
for path in imgs_ordered:
    if path.endswith('.jpg'):
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_list.append(image)

We can vizualize the first image of our list: 

In [ ]:
image_0,image_25,image_50 = img_list[0],img_list[25],img_list[50]
print("Image shape: {}".format(image_0.shape))
visualize(image_0=image_0,
            image_25=image_25,
            image_50=image_50)

## Football Tracker

We first need to create our tracker. This object gathers each and every one of our model: 

```python3
class FootballTracker:
"""Class for the full Football Tracker. Given a list of images, it allows to track and id each player as well as the ball.
It also computes the homography at each given frame, and apply it to each player coordinates. 

Arguments:
    pretrained: Boolean, if the homography and tracking models should be pretrained with our weights or not.
    weights_homo: Path to weight for the homography model
    weights_keypoints: Path to weight for the keypoints model
    shape_in: Shape of the input image
    shape_out: Shape of the ouput image
    conf_tresh: Confidence treshold to keep tracked bouding boxes
    track_buffer: Number of frame to keep in memory for tracking reIdentification
    K: Number of boxes to keep at each frames
    frame_rate: -
Call arguments:
    imgs: List of np.array (images) to track
    split_size: if None, apply the tracking model to the full image. If its an int, the image shape must be divisible by this int.
                We then split the image to create n smaller images of shape (split_size,split_size), and apply the model
                to those.
                We then reconstruct the full images and the full predictions.
    results: list of previous results, to resume tracking
    begin_frame: int, starting frame, if you want to resume tracking 
    verbose: Boolean, to display tracking at each frame or not
    save_tracking_folder: Foler to save the tracking images
    template: Football field, to warp it with the computed homographies on to the saved images
    skip_homo: List of int. e.g.: [4,10] will not compute homography for frame 4 and 10, and reuse the computed homography
                at frame 3 and 9.
"""

    def __init__(
        self,
        pretrained=True,
        weights_homo=None,
        weights_keypoints=None,
        shape_in=512.0,
        shape_out=320.0,
        conf_tresh=0.5,
        track_buffer=30,
        K=100,
        frame_rate=30,
    ):

        self.player_ball_tracker = PlayerBallTracker(
            conf_tresh=conf_tresh, track_buffer=track_buffer, K=K, frame_rate=frame_rate
        )

        self.homo_estimator = HomographyEstimator(
            pretrained=pretrained,
            weights_homo=weights_homo,
            weights_keypoints=weights_keypoints,
            shape_in=shape_in,
            shape_out=shape_out,
        )
```

In [ ]:
tracker = FootballTracker(frame_rate=24.7,track_buffer = 60)

We now only have to call it on our list of images. We manually remove some failed homography estimation, at frame $\in \{25,...,30 \}$ by adding ```skip_homo = [25,26,27,28,29,30]``` into our call.

In [ ]:
trajectories = tracker(img_list,split_size = 512, save_tracking_folder = './test_outputs/',
                        template = template,skip_homo = [25,26,27,28,29,30])

## Saving trajectories

In [ ]:
import json

with open('trajectories.json', 'w') as outfile:
    json.dump(trajectories, outfile)

Let's check the same images as before but with the tracking informations: 

In [ ]:
imgs_ordered = []
for i in range(0,51):
    path = 'test_outputs/test_' + '{:05d}'.format(i) + '.jpg'

    imgs_ordered.append(path)
    
img_list = []
for path in imgs_ordered:
    if path.endswith('.jpg'):
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_list.append(image)

image_0,image_25,image_50 = img_list[0],img_list[25],img_list[50]
print("Image shape: {}".format(image_0.shape))
visualize(image_0=image_0,
            image_25=image_25,
            image_50=image_50)

You can also easily create a movie of the tracking data, and display it: 

In [ ]:
import imageio
import progressbar
with imageio.get_writer('test_outputs/movie.mp4', mode='I',fps=20) as writer:
    for i in progressbar.progressbar(range(0,51)):
        filename = 'test_outputs/test_{:05d}.jpg'.format(i)
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="test_outputs/movie.mp4" type="video/mp4">
</video></div>

##  Process trajectories 

We now have raw trajectories that we need to process. 
Fist, you can do several operations to ensure that the trajectories are good: 

* Delete an id at a certain frame
* Delete an id from every frame
* Merge two ids 
* Add an id at a given frame

These operation are easy to do with some of our functions from ```narya.utils.tracker```: 

```python3
def _remove_coords(traj, ids, frame):
    """Remove the x,y coordinates of an id at a given frame
    Arguments:
        traj: Dict mapping each id to a list of trajectory
        ids: the id to target
        frame: int, the frame we want to remove
    Returns:
        traj: Dict mapping each id to a list of trajectory
    Raises:
    """
    
def _remove_ids(traj, list_ids):
    """Remove ids from a trajectory
    Arguments:
        traj: Dict mapping each id to a list of trajectory
        list_ids: List of id
    Returns:
        traj: Dict mapping each id to a list of trajectory
    Raises:
    """
    
def add_entity(traj, entity_id, entity_traj):
    """Adds a new id with a trajectory 
    Arguments:
        traj: Dict mapping each id to a list of trajectory
        entity_id: the id to add
        entity_traj: the trajectory linked to entity_id we want to add
    Returns:
        traj: Dict mapping each id to a list of trajectory
    Raises:
    """
    
def add_entity_coords(traj, entity_id, entity_traj, max_frame):
    """Add some coordinates to the trajectory of a given id
    Arguments:
        traj: Dict mapping each id to a list of trajectory
        entity_id: the id to target
        entity_traj: List of (x,y,frame) to add to the trajectory of entity_id
        max_frame: int, the maximum number of frame in trajectories
    Returns:
        traj: Dict mapping each id to a list of trajectory
    Raises:
    """
    

def merge_id(traj, list_ids_frame):
    """Merge trajectories of different ids. 
    e.g.: (10,0,110),(12,110,300) will merge the trajectory of 10 between frame 0 and 110 to the 
        trajectory of 12 between frame 110 and 300.
    Arguments:
        traj: Dict mapping each id to a list of trajectory
        list_ids_frame: List of (id,frame_start,frame_end)
    Returns:
        traj: Dict mapping each id to a list of trajectory
    Raises:
    """
    
def merge_2_trajectories(traj1, traj2, id_mapper, max_frame_traj1):
    """Merge 2 dict of trajectories, if you want to merge the results of 2 tracking
    Arguments:
        traj1: Dict mapping each id to a list of trajectory
        traj2: Dict mapping each id to a list of trajectory
        id_mapper: A dict mapping each id in traj1 to id in traj2
        max_frame_traj1: Maximum number of frame in the first trajectory
    Returns:
        traj1: Dict mapping each id to a list of trajectory
    Raises:
    """
```

here, let's assume we don't have to perform any operations, and directly process our trajectoris into a Pandas Dataframe. 

First, we can save our raw trajectory with

```python3
import json

with open('trajectories.json', 'w') as fp:
    json.dump(trajectories, fp)
```

Let's start by padding our trajectories with np.nan and building a dict. for our dataframe: 

In [ ]:
import json

with open('trajectories.json') as json_file:
    trajectories = json.load(json_file)

In [ ]:
from narya.utils.tracker import build_df_per_id

df_per_id = build_df_per_id(trajectories)

We now fill the missing values, and apply a filter to smooth the trajectories: 

In [ ]:
from narya.utils.tracker import fill_nan_trajectories

df_per_id = fill_nan_trajectories(df_per_id,5)

In [ ]:
from narya.utils.tracker import get_full_results

df = get_full_results(df_per_id)

## Trajectory Dataframe

We now have access to a dataframe with for each id, for each frame, the 2D coordinates of the entity.

In [ ]:
df.head()

Finally, this you can save this dataframe using ```df.to_csv('results_df.csv')```

In [ ]:
df.to_csv('results.csv')